In [4]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif,RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import json


In [5]:
res = {
    "scale_Robust": {
        "column_1": [],
        "column_2": [],
        "column_3": [],
        'model': pd.read_csv('scale_Robust.csv')
        
        },
    "scale_min_max": {
        "column_1": [],
        "column_2": [],
        "column_3": [],
        'model': pd.read_csv('scale_min_max.csv')
        },
    
    "scaled_min_sklearn": {
        "column_1": [],
        "column_2": [],
        "column_3": [],
        'model': pd.read_csv('scale_min_sklearn.csv')
        },
    "scale_standard":{
        "column_1": [],
        "column_2": [],
        "column_3": [],
        'model': pd.read_csv('scale_standard.csv')
        },
    
}

In [6]:
def select_beat_feature(i,res,k,x,y):
    selector = SelectKBest(score_func=f_classif, k=k)  # Select top 5 features
    X_new = selector.fit_transform(x, y)
    selected_indices = selector.get_support(indices=True)
    selected_feature_names = list(x.columns[selected_indices])
    res[i]['column_1'] = selected_feature_names

In [7]:
def rfe_feature(i,res,k,x,y):
    model = LogisticRegression(max_iter=1000)
    rfe = RFE(model, n_features_to_select=k)  # Select top 5 features
    X_selected = rfe.fit_transform(x, y)
    selected_feature_indices = rfe.support_
    selected_feature_names = list(x.columns[selected_feature_indices])
    res[i]['column_2'] = selected_feature_names

In [8]:
def random_selection(i,res,k,x,y):
    model = RandomForestClassifier()
    model.fit(x,y)
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    selected_feature_names = [x.columns[i] for i in indices[:k]]
    res[i]['column_3'] = selected_feature_names

In [9]:

for i in res:
    try:
        scaled = res[i]['model']
        x = scaled.drop("Credit_Score", axis=1)
        y = scaled["Credit_Score"]
        select_beat_feature(i,res,12,x,y)
        rfe_feature(i,res,12,x,y)
        random_selection(i,res,12,x,y)
    except:
        print(i,"error")
        continue
# check if all column are same

    
    
    
    



scale_min_max
scale_Robust
scaled_min_sklearn
scale_standard
scaled_log
scaled_log error


In [23]:
loads = res.copy()
commons_1= loads['scale_min_max']['column_1']
commons_2= loads['scale_min_max']['column_2']
commons_3= loads['scale_min_max']['column_3']
for i in loads:
    if 'model' in i:
        del loads[i]['model']
    column_1 = loads[i]['column_1']
    column_2 = loads[i]['column_2']
    column_3 = loads[i]['column_3']
    # extract what is common in all three
    common = list(set(column_1) & set(column_2) & set(column_3))
    loads[i]['common'] = common
    commons_1 = list(set(commons_1) & set(column_1))
    commons_2 = list(set(commons_2) & set(column_2))
    commons_3 = list(set(commons_3) & set(column_3))
loads['common_1'] = list(commons_1)
loads['common_2'] = list(commons_2)
loads['common_3'] = list(commons_3)

common = []
for i in commons_1:
    if i in commons_2 or i in commons_3:
        common.append(i)
loads['common'] = common

print(loads)
with open('feature_selected.json', 'w') as fp:
    json.dump(loads, fp)
        
    